In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
import os

In [10]:
# experimental 
dataa = tf.data.Dataset.from_tensor_slices(range(15))
dataa = dataa.window(5, shift=2, drop_remainder=True)
dataa = dataa.flat_map(lambda window: window.batch(5))
dataa = dataa.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataa = dataa.batch(3).prefetch(1)
for x_batch, y_batch in enumerate(dataa):
    print(x_batch)
    print(y_batch)

0
(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[10, 11, 12, 13],
       [ 6,  7,  8,  9],
       [ 0,  1,  2,  3]])>, <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[11, 12, 13, 14],
       [ 7,  8,  9, 10],
       [ 1,  2,  3,  4]])>)
1
(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 4,  5,  6,  7],
       [ 2,  3,  4,  5],
       [ 8,  9, 10, 11]])>, <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 5,  6,  7,  8],
       [ 3,  4,  5,  6],
       [ 9, 10, 11, 12]])>)


In [7]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [14]:
''.join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [22]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
tokenizer.texts_to_sequences(['Ibrahim'])

[[6, 22, 9, 5, 7, 6, 15]]

In [25]:
max_id = len(tokenizer.word_index)
data_size = tokenizer.document_count

In [36]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = data_size * 90 // 100
data = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [37]:
n_steps = 100
batch_size = 32
window_length = n_steps + 1
data = data.window(window_length, shift=1, drop_remainder=True)
data = data.flat_map(lambda window: window.batch(window_length))
data = data.shuffle(10000).batch(batch_size)
data = data.map(lambda window: (window[:-1], window[1:]))
data = data.map(lambda x_batch, y_batch: (tf.one_hot(x_batch, depth=max_id), y_batch)).prefetch(1)

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(data, epochs=10)

In [38]:
def preprocess(text):
    x = np.array(tokenizer.texts_to_sequences(text)) - 1
    return tf.one_hot(x, max_id)

In [ ]:
x_new = preprocess['what is your nam']
y_pred = np.argmax(model(x_new), axis=1)
tokenizer.sequences_to_texts(y_pred + 1)[0][-1] # 1st sentence, last char

In [ ]:
def next_char(text, temperature=1):
    x_new = preprocess(text)
    y_proba = model(x_new)[0, -1:, :]
    rescaled_pred = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_pred, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]
def complete_text(text, temperature=1, n_char=50):
    for _ in range(n_char):
        text += next_char(text, temperature)
    return text

let's create stateful model

In [45]:
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset2 = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset2 = dataset2.window(window_length, shift=n_steps, drop_remainder=True)
    dataset2 = dataset2.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset2)
dataset2 = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset2 = dataset2.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset2 = dataset2.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset2 = dataset2.prefetch(1)

In [46]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset2, epochs=50,
                    callbacks=[ResetStatesCallback()])

transfering statful model into stateless one for faster operations and less computational power

In [47]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model

c:\Users\ibrah\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
